In [1]:
# # Finding eigenvalues of matrices with neural networks. 
# Modified Script for finding the eigenvectors corresponding to the largest eigenvalue of a matrix with a neural network.
# this is courtesy of Morthen who gave permission before use.

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.reset_default_graph()
# tf.set_random_seed(343)

# import tensorflow as tf
import numpy as np
from matplotlib import cm
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import axes3d
#from lib import compute_dx_dt

np.random.seed(1234)

matrix_size = 6

A = np.random.random_sample(size=(matrix_size,matrix_size))
#A = np.matrix('2 7 3 9 3 0;7 7 1 1 9 7;3 1 8 7 6 6;9 1 7 5 4 8;3 9 6 4 8 7;0 7 6 8 7 9', dtype="float64")
A = (A.T + A)/2.0
start_matrix = A


eigen_vals, eigen_vecs =  np.linalg.eig(A)
print(A)
A = tf.convert_to_tensor(A)
print("A = ", A)

x_0 = tf.convert_to_tensor(np.random.random_sample(size = (1,matrix_size)))
print("x0 = ", x_0)

## The construction phase

num_iter = 10000
num_hidden_neurons = [50]
num_hidden_layers = np.size(num_hidden_neurons)


with tf.variable_scope('dnn'):

    previous_layer = x_0

    for l in range(num_hidden_layers):
        current_layer = tf.layers.dense(previous_layer, num_hidden_neurons[l],activation=tf.nn.sigmoid)
        previous_layer = current_layer

    dnn_output = tf.layers.dense(previous_layer, matrix_size)

with tf.name_scope('loss'):
    print("dnn_output = ", dnn_output)
    
    x_trial = tf.transpose(dnn_output)
    print("x_trial = ", x_trial)
    
    temp1 = (tf.tensordot(tf.transpose(x_trial), x_trial, axes=1)*A)
    temp2 = (1- tf.tensordot(tf.transpose(x_trial), tf.tensordot(A, x_trial, axes=1), axes=1))*np.eye(matrix_size)
    func = tf.tensordot((temp1-temp2), x_trial, axes=1)
    
    print(temp1)
    print(temp2)
    print(func)
    
    func = tf.transpose(func)
    x_trial = tf.transpose(x_trial)
    
    loss = tf.losses.mean_squared_error(func, x_trial)

learning_rate = 0.001

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    traning_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

g_dnn = None

losses = []

with tf.Session() as sess:
    init.run()
    for i in range(num_iter):
        sess.run(traning_op)

        if i % 100 == 0:
            l = loss.eval()
            print("Step:", i, "/",num_iter, "loss: ", l)
            losses.append(l)

    x_dnn = x_trial.eval()
x_dnn = x_dnn.T


# ## Plotting loss over time
"""
plt.plot(losses[:5])
plt.xlabel("Iteration")
plt.ylabel("Loss")

"""

print()
print("Eigenvector NN = \n", (x_dnn/(x_dnn**2).sum()**0.5), "\n")
# Finding eigevalues from eigenvec (largest)
eigen_val_nn = x_dnn.T @ (start_matrix @ x_dnn) / (x_dnn.T @ x_dnn)

print("Eigenvalue NN = \n", eigen_val_nn, "\n \n")
print("Eigenvector analytic = \n", eigen_vecs)
print("\n")
print("Eigenvalues analytic = \n",eigen_vals)


Instructions for updating:
non-resource variables are not supported in the long term


Instructions for updating:
non-resource variables are not supported in the long term


[[0.19151945 0.44928651 0.56059534 0.7790926  0.85655796 0.57086   ]
 [0.44928651 0.80187218 0.83542069 0.87928691 0.50459771 0.46858427]
 [0.56059534 0.83542069 0.37025075 0.46304108 0.45014287 0.40795805]
 [0.7790926  0.87928691 0.46304108 0.61539618 0.43205569 0.25629542]
 [0.85655796 0.50459771 0.45014287 0.43205569 0.31683612 0.63617981]
 [0.57086    0.46858427 0.40795805 0.25629542 0.63617981 0.70458131]]
A =  Tensor("Const:0", shape=(6, 6), dtype=float64)
x0 =  Tensor("Const_1:0", shape=(1, 6), dtype=float64)
dnn_output =  Tensor("dnn/dense_1/BiasAdd:0", shape=(1, 6), dtype=float64)
x_trial =  Tensor("loss/transpose:0", shape=(6, 1), dtype=float64)
Tensor("loss/mul:0", shape=(6, 6), dtype=float64)
Tensor("loss/mul_1:0", shape=(6, 6), dtype=float64)
Tensor("loss/Tensordot_3/MatMul:0", shape=(6, 1), dtype=float64)
Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



/var/folders/t5/qbpdx9511w3d833p3c9qvwrc0000gn/T/ipykernel_55509/316632233.py:47: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  current_layer = tf.layers.dense(previous_layer, num_hidden_neurons[l],activation=tf.nn.sigmoid)
/var/folders/t5/qbpdx9511w3d833p3c9qvwrc0000gn/T/ipykernel_55509/316632233.py:50: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  dnn_output = tf.layers.dense(previous_layer, matrix_size)
2022-12-19 20:56:44.255557: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Step: 0 / 10000 loss:  1.4868191
Step: 100 / 10000 loss:  0.016757816
Step: 200 / 10000 loss:  0.0047608353
Step: 300 / 10000 loss:  0.0017829945
Step: 400 / 10000 loss:  0.00073457125
Step: 500 / 10000 loss:  0.00031603556
Step: 600 / 10000 loss:  0.00013915231
Step: 700 / 10000 loss:  6.215657e-05
Step: 800 / 10000 loss:  2.8055394e-05
Step: 900 / 10000 loss:  1.2775068e-05
Step: 1000 / 10000 loss:  5.865517e-06
Step: 1100 / 10000 loss:  2.7157228e-06
Step: 1200 / 10000 loss:  1.2684895e-06
Step: 1300 / 10000 loss:  5.9803665e-07
Step: 1400 / 10000 loss:  2.8475435e-07
Step: 1500 / 10000 loss:  1.3699488e-07
Step: 1600 / 10000 loss:  6.661951e-08
Step: 1700 / 10000 loss:  3.2749906e-08
Step: 1800 / 10000 loss:  1.6275052e-08
Step: 1900 / 10000 loss:  8.174975e-09
Step: 2000 / 10000 loss:  4.1481103e-09
Step: 2100 / 10000 loss:  2.124234e-09
Step: 2200 / 10000 loss:  1.0974879e-09
Step: 2300 / 10000 loss:  5.715857e-10
Step: 2400 / 10000 loss:  2.997767e-10
Step: 2500 / 10000 loss:  1

In [2]:

print(A)


#
#
#  A = np.matrix('-8 -5  9  -3  8 10; -5 7 2 10 6 -9;9 2 -2 10 -9 -2; -3 10 10 -7 -9 -7 ;8 6 -9 -9 -10 -2;10 -9 -2 -7 -2 -5', dtype="float64")

print(A.shape)

Tensor("Const:0", shape=(6, 6), dtype=float64)
(6, 6)


In [3]:
print(eigen_vecs)
print(start_matrix)

[[-0.40661035 -0.73608462  0.17401828  0.4431149  -0.20662339  0.15336576]
 [-0.48043619 -0.27441836 -0.4469813  -0.46474992  0.18274908 -0.49466518]
 [-0.38109342  0.22082369 -0.1473763  -0.38644582 -0.60274354  0.52119553]
 [-0.42498848  0.3696067  -0.41612106  0.49221703  0.43369944  0.28150689]
 [-0.38403463  0.44071847  0.34772163  0.27641179 -0.35722481 -0.57737445]
 [-0.36127426  0.05717523  0.673898   -0.34588329  0.49487878  0.21802093]]
[[0.19151945 0.44928651 0.56059534 0.7790926  0.85655796 0.57086   ]
 [0.44928651 0.80187218 0.83542069 0.87928691 0.50459771 0.46858427]
 [0.56059534 0.83542069 0.37025075 0.46304108 0.45014287 0.40795805]
 [0.7790926  0.87928691 0.46304108 0.61539618 0.43205569 0.25629542]
 [0.85655796 0.50459771 0.45014287 0.43205569 0.31683612 0.63617981]
 [0.57086    0.46858427 0.40795805 0.25629542 0.63617981 0.70458131]]
